In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import numpy as np
import lightgbm as lgb
from catboost import CatBoostClassifier

In [11]:
# Load data
df = pd.read_csv("./data/train.csv")

# Features and target
X = df.drop(columns=["song_popularity", "id"])
y = df["song_popularity"]

# Identify types
num_cols = ["song_duration_ms","acousticness","danceability","energy",
            "instrumentalness","liveness","loudness","speechiness",
            "tempo","audio_valence"]
cat_cols = ["key","audio_mode","time_signature"]

# Preprocessing
# --- Feature Engineering ---
def feature_engineering(df):
    df = df.copy()
    df["energy_dance"] = df["energy"] * df["danceability"]
    df["speech_instr"] = df["speechiness"] * (1 - df["instrumentalness"])
    df["log_duration"] = np.log1p(df["song_duration_ms"].fillna(df["song_duration_ms"].median()))
    df["log_tempo"] = np.log1p(df["tempo"].fillna(df["tempo"].median()))
    return df

feat_engineer = FunctionTransformer(feature_engineering)

# Update feature lists with engineered features
extra_num_cols = ["energy_dance","speech_instr","log_duration","log_tempo"]

num_transformer = Pipeline([
    ("imputer", KNNImputer(n_neighbors=5)),
    ("scaler", StandardScaler())
])
cat_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_transformer, num_cols + extra_num_cols),
    ("cat", cat_transformer, cat_cols)
])




In [12]:
xgb_model = xgb.XGBClassifier(
    n_estimators=800,
    learning_rate=0.05,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="gpu_hist",      # GPU training
    predictor="gpu_predictor",   # GPU inference
    eval_metric="logloss"
)

lgb_model = lgb.LGBMClassifier(
    n_estimators=800,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    device="gpu"                 # GPU mode
)

cat_model = CatBoostClassifier(
    iterations=800,
    learning_rate=0.05,
    depth=10,
    task_type="GPU",             # GPU mode
    verbose=0
)

# --- Ensemble ---
ensemble = VotingClassifier(
    estimators=[("xgb", xgb_model), ("lgb", lgb_model), ("cat", cat_model)],
    voting="soft"
)

# --- Final Pipeline ---
clf = Pipeline([
    ("features", feat_engineer),
    ("preprocessor", preprocessor),
    ("model", ensemble)
])

# Cross-validation
scores = cross_val_score(clf, X, y, cv=5, scoring="accuracy")
print("CV Accuracy:", scores.mean(), "±", scores.std())

# Train on full dataset
clf.fit(X, y)

/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:05:06] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:05:06] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 8746, number of negative: 15254
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3606
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 32
[LightGBM] [Info] Using GPU Device: NVIDIA RTX A4500, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (0.46 MB) transferred to GPU in 0.001164 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.364417 -> initscore=-0.556245
[LightGBM] [Info] Start training from score -0.556245


/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/xgboost/core.py:2676: UserWarning: [00:07:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/xgboost/core.py:729: UserWarning: [00:07:20] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)
/home/anirudhgupt

...

/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/xgboost/core.py:2676: UserWarning: [00:18:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


CV Accuracy: 0.6223666666666666 ± 0.003668787265568799


/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:18:25] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:18:25] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "predictor" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 10932, number of negative: 19068
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3606
[LightGBM] [Info] Number of data points in the train set: 30000, number of used features: 32
[LightGBM] [Info] Using GPU Device: NVIDIA RTX A4500, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (0.57 MB) transferred to GPU in 0.001084 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.364400 -> initscore=-0.556317
[LightGBM] [Info] Start training from score -0.556317


,steps,"[('features', ...), ('preprocessor', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,func,<function fea...x73587d931f80>
,inverse_func,None
,validate,False
,accept_sparse,False
,check_inverse,True
,feature_names_out,None
,kw_args,None


In [13]:
# Predict on test
test = pd.read_csv("./data/test.csv")
test_ids = test["id"]
X_test = test.drop(columns=["id"])
test_preds = clf.predict(X_test)

# Save submission
submission = pd.DataFrame({
    "id": test_ids,
    "song_popularity": test_preds
})
submission.to_csv("submission_2.csv", index=False)

print("✅ submission.csv file created!")

✅ submission.csv file created!


/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/xgboost/core.py:2676: UserWarning: [00:34:00] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
/home/anirudhgupta/main/umc301/kaggle1/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
